In [2]:
# ! pip install ctransformers[cuda]

In [6]:
# ! pip install langchain
# ! pip install faiss_cpu
# ! pip install sentence_transformers

In [1]:
from langchain.llms import CTransformers
import torch
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
# from langchain.llms import CTransformers
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import sys

In [10]:
llm = CTransformers(model='TheBloke/Llama-2-7B-GGML', model_file='llama-2-7b.ggmlv3.q2_K.bin',
                     max_new_tokens=512,
                    temperature=0.1,gpu_layers=50)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b.ggmlv3.q2_K.bin:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

In [4]:


DB_FAISS_PATH = "vectorstore/db_faiss"
loader = CSVLoader(file_path="/content/2019.csv")
data = loader.load()

In [5]:
# Split the text into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

print(len(text_chunks))

# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

# COnverting the text Chunks into embeddings and saving the embeddings into FAISS Knowledge Base
docsearch = FAISS.from_documents(text_chunks, embeddings)

docsearch.save_local(DB_FAISS_PATH)

156


In [11]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=docsearch.as_retriever())

In [7]:
while True:
    chat_history = []
    #query = "What is the value of  GDP per capita of Finland provided in the data?"
    query = input(f"Input Prompt: ")
    if query == 'exit':
        print('Exiting')
        sys.exit()
    if query == '':
        continue
    result = qa({"question":query, "chat_history":chat_history})
    print("Response: ", result['answer'])

Input Prompt: What is the value of GDP per capita of Finland provided in the data?
Response:   I don't know the answer to this question, as the data does not provide the exact value of GDP per capita for Finland.
Input Prompt: exit
Exiting


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=docsearch.as_retriever())

query = "What is the value of GDP per capita of India provided in the data?"

chat_history = []
result = qa({"question": query, "chat_history": chat_history})
print("Response: ", result['answer'])